In [6]:
import tweepy
from datetime import datetime
import csv
import os
import json
# import daemon
from config import getConfig

def test(api):
    # Test api
    public_tweets = api.home_timeline()
    print(len(public_tweets))


def load_config_tk(filename):
    with open(filename, "r") as f:
        consumer_key, consumer_secret, access_token, access_token_secret = [line.strip() for line in f.readlines()]
        print(consumer_key, consumer_secret, access_token, access_token_secret)
        return consumer_key, consumer_secret, access_token, access_token_secret

def loadFilter(filename):
    with open(filename, "r") as filterJson:
        j = json.load(filterJson)
        return j
    
def sperateBoundbox(degree):
    minlong = boundbox[0]
    minlat = boundbox[1]
    maxlong = boundbox[2]
    maxlat = boundbox[3]

    degree = 1000

    boxs = []
    detal_lat = (maxlat - minlat) / degree
    detal_long = (maxlong - minlong) / degree

    l1 = minlong
    l2 = minlat

    for x in range(degree):
        l1 = minlong
        l2 += detal_lat
        l4 = l2 + detal_lat
        for y in range(degree):
            l1 += detal_long
            l3 = l1 + detal_long
            box = [l1, l2, l3, l4]
            boxs.append(box)
    return boxs


def create_csv(filename):
    print("Start running on", filename)
    with open(filename, "w") as outfile:
        writer = csv.writer(outfile)
        if getAllTweets:
            writer.writerow(title)
        else:
            writer.writerow(jsonKeys)


def write_csv(data):
    filename = datetime.now().strftime("%Y%m%d-%H") + ".csv"
    if os.path.isfile(filename):
        pass
    else:
        create_csv(filename)
    with open(filename, "a") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(data)


# override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        contents = []
        if ignoreGeoTag or status.geo is not None:
            if getAllTweets:
                # get all tweets
                for attr in dir(status):
                    if not attr.startswith("__"):
                        contents.append(getattr(status, attr))
            else:
                # only essentail data 
                tweetJson = status._json
                for k in jsonKeys:
                    contents.append(tweetJson[k])
            # print(status.text)
            write_csv(contents)
    def on_error(self, status_code):
        if status_code == 420:
            print("Error: Status Code 420")
            # returning False in on_error disconnects the stream
            return False

        # returning non-False reconnects the stream, with backoff.

########
# setup config
tokenPath = '../HPRC-Python3.7-Example/token.tk'
consumer_key, consumer_secret, access_token, access_token_secret = load_config_tk(tokenPath)

title = getConfig()['title']
jsonKeys = getConfig()['jsonKeys']

#setup boundbox
filters = loadFilter("./filter.json")
# got the real title we want from twitter api
#setup geoconfig: if Ture, get every tweet; if False, only tweet with geotag will write down
ignoreGeoTag = False
# setup if all the twitter data is needed, not recommend cuz it may cause lots of unnecessary data
getAllTweets = False
########

def main():
    # setup api
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    
    # test
    test(api)
    
    # with daemon.DaemonContext():
    print("Start running on", datetime.now().strftime("%Y%m%d-%H%M%S"))
    myStreamListener = MyStreamListener()
    myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener)
    myStream.filter(locations=filters.locations.boundbox, is_async=False)



if __name__ == "__main__":
    print("Start running...")
    # main()

injAlTM28wUu5L0qKnZQY0TI2 I6FlkCViiAC1eYxx7aUHZbBqJUt4JE3rnIhZKHZkDAYZmJwN0w 1140116930247811073-og3G7DJp4yPKlmUYoVTdvIBKx9EECW XFlbHd6H7oOd2kae2xWlomNbpPKVkX3jpGGpf6fPzjJqG
Start running...
